In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from scipy import stats as st
from random import randrange

In [2]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = 200
data = pd.read_csv('clean_pan.csv', index_col='CASEID')


In [3]:
data.shape

(23233, 80)

In [4]:
data = data.drop(['Unnamed: 0'], axis=1)

In [5]:
y = data['PAN_FISTULA']
X = data.drop(['PAN_FISTULA'], axis=1)

In [6]:
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
fold_no=0
for train_index, test_index in skf.split(data, y):
    train = data.loc[train_index,:]
    test = data.loc[test_index,:]
    train_filename = 'train' + str(fold_no) + '.csv'
    test_filename = 'test' + str(fold_no) + '.csv' 
    train.to_csv('splits/' + train_filename, index=False)
    test.to_csv('splits/' + test_filename, index=False) 
    fold_no += 1

In [7]:
d = {}
for x in range(0,5):
    d['train{}'.format(x)] = pd.read_csv('splits/train{}.csv'.format(x), low_memory=False)
    d['test{}'.format(x)] = pd.read_csv('splits/test{}.csv'.format(x), low_memory=False)

In [8]:
dX_train = []
dy_train = []
for x in d:
    if 'train' in x:
        dX_train.append(d[x].drop(columns=['PAN_FISTULA'], axis=1))
        dy_train.append(d[x]['PAN_FISTULA'])

In [9]:
dX_test = []
dy_test = []
for x in d:
    if 'test' in x:
        dX_test.append(d[x].drop(columns=['PAN_FISTULA'], axis=1))
        dy_test.append(d[x]['PAN_FISTULA'])

rfpreds = []
xgbpreds = []
model = RandomForestClassifier(n_estimators=1250, min_samples_split=2, min_samples_leaf=8, max_features='auto', max_depth=20, bootstrap=True)
model2 = XGBClassifier(n_estimators=50, subsample=0.6, min_child_weight=10, max_depth=6, learning_rate=0.1, colsample_bytree=0.8)
for X, y, X_test in zip(dX_train, dy_train, dX_test):
    model.fit(X, y)
    model2.fit(X, y)
    rfpreds.append(model.predict_proba(X_test))
    xgbpreds.append(model2.predict_proba(X_test))

%store rfpreds
%store xgbpreds

for x in range(0,5):
    print(roc_auc_score(dy_test[x], rfpreds[x][:,1]))
    

for x in range(0,5):
    print(roc_auc_score(dy_test[x], xgbpreds[x][:,1]))

In [10]:
from tensorflow import keras
input_shape = [X.shape[1]]
model4 = keras.models.Sequential()
model4.add(keras.layers.Flatten(input_shape=input_shape))
model4.add(keras.layers.BatchNormalization())
for _ in range(2):
    model4.add(keras.layers.Dense(1000))
    model4.add(keras.layers.BatchNormalization())
    model4.add(keras.layers.Dropout(0.8))
    model4.add(keras.layers.Activation("relu"))
model4.add(keras.layers.Dense(1, activation="sigmoid"))

opt = keras.optimizers.Adam(learning_rate=3e-4)

metrics = [keras.metrics.Recall(name='Sensitivity'), keras.metrics.TrueNegatives(name='tn'), keras.metrics.AUC(name='auc'), keras.metrics.AUC(name='prc', curve='PR')]

model4.compile(
    optimizer=opt,
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=metrics,)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.00001,
    restore_best_weights=True,)


In [11]:
annpreds = []
for X, y, X_test in zip(dX_train, dy_train, dX_test):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)
    model4.fit(X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping])
    annpreds.append(model4.predict(X_test))

Epoch 1/200
28/28 [==============================] - 3s 38ms/step - loss: 0.8784 - Sensitivity: 0.3826 - tn: 7698.0000 - auc: 0.5346 - prc: 0.1790 - val_loss: 0.6517 - val_Sensitivity: 0.5972 - val_tn: 2647.0000 - val_auc: 0.6806 - val_prc: 0.3030
Epoch 2/200
28/28 [==============================] - 0s 11ms/step - loss: 0.6375 - Sensitivity: 0.1447 - tn: 10493.0000 - auc: 0.5625 - prc: 0.1954 - val_loss: 0.5294 - val_Sensitivity: 0.2284 - val_tn: 3550.0000 - val_auc: 0.6944 - val_prc: 0.3123
Epoch 3/200
28/28 [==============================] - 0s 12ms/step - loss: 0.6014 - Sensitivity: 0.1470 - tn: 10654.0000 - auc: 0.5939 - prc: 0.2180 - val_loss: 0.4923 - val_Sensitivity: 0.1731 - val_tn: 3652.0000 - val_auc: 0.6983 - val_prc: 0.3152
Epoch 4/200
28/28 [==============================] - 0s 12ms/step - loss: 0.5895 - Sensitivity: 0.1756 - tn: 10652.0000 - auc: 0.6043 - prc: 0.2313 - val_loss: 0.4869 - val_Sensitivity: 0.1995 - val_tn: 3600.0000 - val_auc: 0.6997 - val_prc: 0.3161
Epoch

In [12]:
ann_score = []
for x in range(0,5):
    ann_score.append(roc_auc_score(dy_test[x], annpreds[x]))
ann_score

[0.7230773695356694,
 0.7279776436808083,
 0.7445381490794634,
 0.7465062050508798,
 0.7479201841048017]

model.fit(dX_train[0], dy_train[0])
def rf_feat_importance(model, X):
    return pd.DataFrame({'cols':X.columns, 'imp':model.feature_importances_}
                       ).sort_values('imp', ascending=False)
fi = rf_feat_importance(model, X)
fi[:10]
def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

plot_fi(fi[:20]);

model2.fit(dX_train[0], dy_train[0])
fi = rf_feat_importance(model2, X)
plot_fi(fi[:20]);

In [13]:
lrpreds = []
model3 = LogisticRegression()
for X, y, X_test in zip(dX_train, dy_train, dX_test):
    model3.fit(X, y)
    lrpreds.append(model3.predict_proba(X_test))
lr_score = []
for x in range(0,5):
    lr_score.append(roc_auc_score(dy_test[x], lrpreds[x][:,1]))
lr_score

/home/kchen/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kchen/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

[0.7033941714706838,
 0.7045845681002759,
 0.6989298211679473,
 0.7052373866157211,
 0.706601778639608]

lr_ci = stats.norm.interval(0.95, loc=np.mean(lr_score), scale=np.std(lr_score))
print(lr_ci)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)

def evaluate(model1, X, y):
    ppreds = model1.predict_proba(X)
    ppreds = ppreds[:,1]
    pscore = roc_auc_score(y, ppreds)
    print('AUC', pscore)
model = RandomForestClassifier()
model.fit(X_train, y_train)
evaluate(model, X_valid, y_valid)



def rf_feat_importance(model, X):
    return pd.DataFrame({'cols':X.columns, 'imp':model.feature_importances_}
                       ).sort_values('imp', ascending=False)
fi = rf_feat_importance(model, X)
fi[:10]
def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(8,4), legend=False)

plot_fi(fi[:10]);

In [14]:
ann_mean = np.mean(ann_score)
ann_confidence = st.t.interval(0.95, len(ann_score)-1, loc=ann_mean, scale=st.sem(ann_score))

print('Neural Network:', round(ann_mean,3), '('+str(round(ann_confidence[0],3))+'-'+str(round(ann_confidence[1],3))+')')


Neural Network: 0.738 (0.724-0.752)


In [15]:
lr_mean = np.mean(lr_score)
lr_confidence = st.t.interval(0.95, len(lr_score)-1, loc=lr_mean, scale=st.sem(lr_score))

print('Logistic Regression:', round(lr_mean,3), '('+str(round(lr_confidence[0],3))+'-'+str(round(lr_confidence[1],3))+')')


Logistic Regression: 0.704 (0.7-0.707)


In [16]:
ann_prc = []
for x in range(0,5):
    ann_prc.append(average_precision_score(dy_test[x], annpreds[x]))

ann_prc_mean = np.mean(ann_prc)
ann_prc_confidence = st.t.interval(0.95, len(ann_prc)-1, loc=ann_prc_mean, scale=st.sem(ann_prc))

print('Neural Network:', round(ann_prc_mean,3), '('+str(round(ann_prc_confidence[0],3))+'-'+str(round(ann_prc_confidence[1],3))+')')
lr_prc = []
for x in range(0,5):
    lr_prc.append(average_precision_score(dy_test[x], lrpreds[x][:,1]))

lr_prc_mean = np.mean(lr_prc)
lr_prc_confidence = st.t.interval(0.95, len(lr_prc)-1, loc=lr_prc_mean, scale=st.sem(lr_prc))

print('Logistic Regression:', round(lr_prc_mean,3), '('+str(round(lr_prc_confidence[0],3))+'-'+str(round(lr_prc_confidence[1],3))+')')
with open('pan_results.txt', 'w') as f:
    f.write('Logistic Regression: '+str(round(lr_mean,3))+' ('+str(round(lr_confidence[0],3))+'-'+str(round(lr_confidence[1],3))+')\n')
    f.write('Neural Network: '+str(round(ann_mean,3))+' ('+str(round(ann_confidence[0],3))+'-'+str(round(ann_confidence[1],3))+')\n')
    f.write('AUPRC\n')
    f.write('Logistic Regression: '+str(round(lr_prc_mean,3))+' ('+str(round(lr_prc_confidence[0],3))+'-'+str(round(lr_prc_confidence[1],3))+')\n')
    f.write('Neural Network: '+str(round(ann_prc_mean,3))+' ('+str(round(ann_prc_confidence[0],3))+'-'+str(round(ann_prc_confidence[1],3))+')')

Neural Network: 0.367 (0.353-0.381)
Logistic Regression: 0.318 (0.299-0.337)


In [17]:
for x in range(0,5):
    print(len(annpreds[x]))

4647
4647
4647
4646
4646


In [18]:
pan_ann_tpr = []
pan_ann_fpr = []
for x in range(0,5):
    fpr, tpr, thresholds = roc_curve(dy_test[x], annpreds[x])
    pan_ann_tpr.append(tpr)
    pan_ann_fpr.append(fpr)
pan_ann_tpr_array = [np.array(x) for x in pan_ann_tpr]
mean_pan_ann_tpr = [np.mean(k) for k in zip(*pan_ann_tpr_array)]
pan_ann_fpr_array = [np.array(x) for x in pan_ann_fpr]
mean_pan_ann_fpr = [np.mean(k) for k in zip(*pan_ann_fpr_array)]
%store mean_pan_ann_tpr
%store mean_pan_ann_fpr


Stored 'mean_pan_ann_tpr' (list)
Stored 'mean_pan_ann_fpr' (list)


In [19]:
for x in range(0,5):
    print(len(pan_ann_fpr[x]))
    

1130
1180
1116
1159
1078


In [20]:
for x in range(0,5):
    pan_ann_tpr[x] = np.random.permutation(pan_ann_tpr[x])
pan_ann_tpr = [x[:1000] for x in pan_ann_tpr]
for x in range(0,5):
    pan_ann_tpr[x] = sorted(pan_ann_tpr[x])
for x in range(0,5):
    pan_ann_fpr[x] = np.random.permutation(pan_ann_fpr[x])
pan_ann_fpr = [x[:1000] for x in pan_ann_fpr]
for x in range(0,5):
    pan_ann_fpr[x] = sorted(pan_ann_fpr[x])

In [21]:
pan_ann_tpr_array = [np.array(x) for x in pan_ann_tpr]
mean_pan_ann_tpr = [np.mean(k) for k in zip(*pan_ann_tpr_array)]
pan_ann_fpr_array = [np.array(x) for x in pan_ann_fpr]
mean_pan_ann_fpr = [np.mean(k) for k in zip(*pan_ann_fpr_array)]
%store mean_pan_ann_tpr
%store mean_pan_ann_fpr

Stored 'mean_pan_ann_tpr' (list)
Stored 'mean_pan_ann_fpr' (list)


In [22]:
pan_ann_tpr = []
pan_ann_fpr = []
for x in range(0,5):
    fpr, tpr, _ = roc_curve(dy_test[x], annpreds[x])
    pan_ann_tpr.append(tpr)
    pan_ann_fpr.append(fpr)
for x in range(0,5):
    diff = len(pan_ann_tpr[x]) - 1000
    for _ in range(diff):
        ind = randrange(len(pan_ann_tpr[x]))
        pan_ann_tpr[x] = np.delete(pan_ann_tpr[x],ind)
for x in range(0,5):
    diff = len(pan_ann_fpr[x]) - 1000
    for _ in range(diff):
        ind = randrange(len(pan_ann_fpr[x]))
        pan_ann_fpr[x] = np.delete(pan_ann_fpr[x],ind)


mean_pan_ann_tpr = [np.mean(k) for k in zip(*pan_ann_tpr)]

mean_pan_ann_fpr = [np.mean(k) for k in zip(*pan_ann_fpr)]
%store mean_pan_ann_tpr
%store mean_pan_ann_fpr

Stored 'mean_pan_ann_tpr' (list)
Stored 'mean_pan_ann_fpr' (list)


In [23]:
pan_lr_tpr = []
pan_lr_fpr = []
for x in range(0,5):
    fpr, tpr, _ = roc_curve(dy_test[x], lrpreds[x][:,1])
    pan_lr_tpr.append(tpr)
    pan_lr_fpr.append(fpr)
for x in range(0,5):
    diff = len(pan_lr_tpr[x]) - 1138
    for _ in range(diff):
        ind = randrange(len(pan_lr_tpr[x]))
        pan_lr_tpr[x] = np.delete(pan_lr_tpr[x],ind)

for x in range(0,5):
    diff = len(pan_lr_fpr[x]) - 1138
    for _ in range(diff):
        ind = randrange(len(pan_lr_fpr[x]))
        pan_lr_fpr[x] = np.delete(pan_lr_fpr[x],ind)


mean_pan_lr_tpr = [np.mean(k) for k in zip(*pan_lr_tpr)]

mean_pan_lr_fpr = [np.mean(k) for k in zip(*pan_lr_fpr)]
%store mean_pan_lr_tpr
%store mean_pan_lr_fpr

Stored 'mean_pan_lr_tpr' (list)
Stored 'mean_pan_lr_fpr' (list)


In [24]:
pan_lr_rec = []
pan_lr_prec = []
for x in range(0,5):
    prec, rec, _ = precision_recall_curve(dy_test[x], lrpreds[x][:,1])
    pan_lr_rec.append(rec)
    pan_lr_prec.append(prec)

for x in range(0,5):
    diff = len(pan_lr_rec[x]) - 4523
    for _ in range(diff):
        ind = randrange(len(pan_lr_rec[x]))
        pan_lr_rec[x] = np.delete(pan_lr_rec[x],ind)

for x in range(0,5):
    diff = len(pan_lr_prec[x]) - 4523
    for _ in range(diff):
        ind = randrange(len(pan_lr_prec[x]))
        pan_lr_prec[x] = np.delete(pan_lr_prec[x],ind)

mean_pan_lr_rec = [np.mean(k) for k in zip(*pan_lr_rec)]

mean_pan_lr_prec = [np.mean(k) for k in zip(*pan_lr_prec)]
%store mean_pan_lr_rec
%store mean_pan_lr_prec

Stored 'mean_pan_lr_rec' (list)
Stored 'mean_pan_lr_prec' (list)


In [25]:
pan_ann_rec = []
pan_ann_prec = []
for x in range(0,5):
    prec, rec, _ = precision_recall_curve(dy_test[x], annpreds[x])
    pan_ann_rec.append(rec)
    pan_ann_prec.append(prec)

for x in range(0,5):
    diff = len(pan_ann_rec[x]) - 4523
    for _ in range(diff):
        ind = randrange(len(pan_ann_rec[x]))
        pan_ann_rec[x] = np.delete(pan_ann_rec[x],ind)

for x in range(0,5):
    diff = len(pan_ann_prec[x]) - 4523
    for _ in range(diff):
        ind = randrange(len(pan_ann_prec[x]))
        pan_ann_prec[x] = np.delete(pan_ann_prec[x],ind)

mean_pan_ann_rec = [np.mean(k) for k in zip(*pan_ann_rec)]

mean_pan_ann_prec = [np.mean(k) for k in zip(*pan_ann_prec)]
%store mean_pan_ann_rec
%store mean_pan_ann_prec

Stored 'mean_pan_ann_rec' (list)
Stored 'mean_pan_ann_prec' (list)


pan_lr_rec = []
pan_lr_prec = []
for x in range(0,5):
    prec, rec, _ = precision_recall_curve(dy_test[x], lrpreds[x][:,1])
    pan_lr_rec.append(rec)
    pan_lr_prec.append(prec)

for x in range(0,5):
    pan_lr_rec[x] = np.random.permutation(pan_lr_rec[x])
pan_lr_rec = [x[:1000] for x in pan_lr_rec]
for x in range(0,5):
    pan_lr_rec[x] = sorted(pan_lr_rec[x])
for x in range(0,5):
    pan_lr_prec[x] = np.random.permutation(pan_lr_prec[x])
pan_lr_prec = [x[:1000] for x in pan_lr_prec]
for x in range(0,5):
    pan_lr_prec[x] = sorted(pan_lr_prec[x])
pan_lr_rec_array = [np.array(x) for x in pan_lr_rec]
mean_pan_lr_rec = [np.mean(k) for k in zip(*pan_lr_rec_array)]
pan_lr_prec_array = [np.array(x) for x in pan_lr_prec]
mean_pan_lr_prec = [np.mean(k) for k in zip(*pan_lr_prec_array)]
%store mean_pan_lr_rec
%store mean_pan_lr_prec

from matplotlib import pyplot as plt

plt.plot(mean_pan_ann_rec, mean_pan_ann_prec)